# Transferability Metrics

TJ Kim <br/>
2/4/21

Updated <br/>
2/21/21

#### Objective: 
To the existing (and functioning) FGSM attack, add measurements of transferability between different networks.
This will help us compare as well. 

#### Transferability Metrics
- Size of input gradient
- Gradient Alignment
- Variance of Loss

#### Other Metrics
- Confidence of attack (Loss towards target)
- L2 Distance from original point


In [1]:
cd '/home/ubuntu/FedAtk/' 

/home/ubuntu/FedAtk


### Load Relevant Libraries and Modules

Load the relevant libraries for the federated learning code.

In [2]:
import torch

# Personal NN
# from transfer_attacks.Personalized_NN import *
from transfer_attacks.Transferer import *
# Import the Rest

### Load and Run the System

In [3]:
client_idx = 0
victim_idxs = [0,1,2,3,4,5,6,7]

# Generate NN and Victims
transferer = Transferer(filename = 'exp2_neck2_head3')
transferer.generate_advNN(client_idx = client_idx)
transferer.generate_victims(client_idxs = victim_idxs)

Loading  all_data_12_niid_0_keep_0_train_9.json
Loading  all_data_20_niid_0_keep_0_train_9.json
Loading  all_data_11_niid_0_keep_0_train_9.json
Loading  all_data_18_niid_0_keep_0_train_9.json


/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
transferer.generate_xadv(atk_type = "ifsgm")
transferer.send_to_victims(victim_idxs)

In [5]:
# Run the empirical metrics
transferer.check_empirical_metrics(orig_flag = True, batch_size = 100)
#print(transferer.metric_variance)
#print(transferer.metric_ingrad)
print(transferer.metric_alignment[3])

tensor(1.1590)


In [6]:
for i in range(8):
    print(torch.argmax(transferer.victims[i](transferer.x_orig),dim=1))

tensor([ 2, 19, 29,  7, 32,  3,  3, 36,  3,  1], device='cuda:0')
tensor([ 2, 19, 29,  7, 32,  3,  3, 26,  3,  1], device='cuda:0')
tensor([ 2, 39, 29,  7, 10,  3,  3, 26,  3,  1], device='cuda:0')
tensor([ 2, 30, 29,  7, 22,  3,  3, 36,  3,  1], device='cuda:0')
tensor([ 2, 30, 29,  7, 23,  3,  3,  9,  3,  1], device='cuda:0')
tensor([ 2,  3, 29,  7, 32,  3,  3, 26,  3,  1], device='cuda:0')
tensor([ 2, 39, 29,  7, 23,  3,  3, 26,  3,  1], device='cuda:0')
tensor([ 2, 39, 29,  7, 32,  3,  3, 36,  3,  1], device='cuda:0')


In [7]:
for i in range(8):
    print(torch.argmax(transferer.victims[i](transferer.x_adv),dim=1))

tensor([20, 20, 20, 20, 20, 20, 20, 20, 20, 20], device='cuda:0')
tensor([23, 17,  8, 41, 20,  0, 20, 26, 20, 20], device='cuda:0')
tensor([20, 17, 27, 20, 46,  3, 46, 20, 20, 34], device='cuda:0')
tensor([33, 23, 20, 20, 20, 24, 20, 20,  5, 20], device='cuda:0')
tensor([33, 20, 33, 20, 20,  0, 14, 20,  5, 20], device='cuda:0')
tensor([46, 20, 33, 15, 27, 24, 15, 23, 11, 20], device='cuda:0')
tensor([20, 20, 20,  5, 20,  0, 20, 23, 20, 20], device='cuda:0')
tensor([23, 17, 25, 20, 27,  7,  4, 20,  4, 20], device='cuda:0')


In [8]:
for j in range(8):
    print("\n NN", j)
    print("orig_acc_transfers: ",transferer.orig_acc_transfers[j])
    print("orig_similarities: ",transferer.orig_similarities[j])
    print("orig_target_hit:",transferer.orig_target_hit[j])
    print("adv_acc_transfers:",transferer.adv_acc_transfers[j])
    print("adv_similarities:",transferer.adv_similarities[j])
    print("adv_target_hit:",transferer.adv_target_hit[j])
    print(transferer.metric_alignment[j])


 NN 0
orig_acc_transfers:  tensor(1., device='cuda:0')
orig_similarities:  tensor(1., device='cuda:0')
orig_target_hit: tensor(0., device='cuda:0')
adv_acc_transfers: tensor(0., device='cuda:0')
adv_similarities: tensor(1., device='cuda:0')
adv_target_hit: tensor(1., device='cuda:0')
tensor(0.0005)

 NN 1
orig_acc_transfers:  tensor(0.9000, device='cuda:0')
orig_similarities:  tensor(0.9000, device='cuda:0')
orig_target_hit: tensor(0., device='cuda:0')
adv_acc_transfers: tensor(0., device='cuda:0')
adv_similarities: tensor(0.4000, device='cuda:0')
adv_target_hit: tensor(0.4000, device='cuda:0')
tensor(1.2173)

 NN 2
orig_acc_transfers:  tensor(0.7000, device='cuda:0')
orig_similarities:  tensor(0.7000, device='cuda:0')
orig_target_hit: tensor(0., device='cuda:0')
adv_acc_transfers: tensor(0.1000, device='cuda:0')
adv_similarities: tensor(0.4000, device='cuda:0')
adv_target_hit: tensor(0.4000, device='cuda:0')
tensor(1.3760)

 NN 3
orig_acc_transfers:  tensor(0.8000, device='cuda:0')
o